In [11]:
import warnings
warnings.filterwarnings('ignore')
 

In [12]:
from __future__ import division

import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

import optuna

In [13]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

In [16]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    param = {
        'silent': 1,
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
    }

    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
        param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-auc')
    bst = xgb.train(param, dtrain, evals=[(dtest, 'validation')], callbacks=[pruning_callback])
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return accuracy

In [17]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                                direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial)

[0]	validation-auc:0.871242
[1]	validation-auc:0.871242
[2]	validation-auc:0.871242
[3]	validation-auc:0.871242
[4]	validation-auc:0.871242
[5]	validation-auc:0.871242
[6]	validation-auc:0.871242
[7]	validation-auc:0.871242
[8]	validation-auc:0.871242
[9]	validation-auc:0.871242


[I 2019-11-01 16:01:05,645] Finished trial#0 resulted in value: 0.8741258741258742. Current best value is 0.8741258741258742 with parameters: {'booster': 'dart', 'lambda': 0.00019212467951018207, 'alpha': 2.411601375921556e-08, 'max_depth': 1, 'eta': 0.00018604254675013283, 'gamma': 1.4712627517184022e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 4.1506713452750165e-05, 'skip_drop': 4.652548820330033e-08}.


[0]	validation-auc:0.930419
[1]	validation-auc:0.949302
[2]	validation-auc:0.947865
[3]	validation-auc:0.952586
[4]	validation-auc:0.94725
[5]	validation-auc:0.948686
[6]	validation-auc:0.949713
[7]	validation-auc:0.952381
[8]	validation-auc:0.953202
[9]	validation-auc:0.954844


[I 2019-11-01 16:01:05,760] Finished trial#1 resulted in value: 0.9020979020979021. Current best value is 0.9020979020979021 with parameters: {'booster': 'gblinear', 'lambda': 1.3185031003662474e-07, 'alpha': 0.7318297293890196}.


[0]	validation-auc:0.925803
[1]	validation-auc:0.925803
[2]	validation-auc:0.925803
[3]	validation-auc:0.925803
[4]	validation-auc:0.925803
[5]	validation-auc:0.925803
[6]	validation-auc:0.925803
[7]	validation-auc:0.925803
[8]	validation-auc:0.925803
[9]	validation-auc:0.925803


...

[0]	validation-auc:0.980518
[1]	validation-auc:0.980518
[2]	validation-auc:0.980518
[3]	validation-auc:0.980518
[4]	validation-auc:0.980518
[5]	validation-auc:0.980518
[6]	validation-auc:0.980518
[7]	validation-auc:0.980518


[I 2019-11-01 16:01:29,065] Setting status of trial#98 as TrialState.PRUNED. Trial was pruned at iteration 8.


[0]	validation-auc:0.955993
[1]	validation-auc:0.956409
[2]	validation-auc:0.958906
[3]	validation-auc:0.958489
[4]	validation-auc:0.959426
[5]	validation-auc:0.958489


[I 2019-11-01 16:01:29,379] Setting status of trial#99 as TrialState.PRUNED. Trial was pruned at iteration 6.


FrozenTrial(number=8, state=<TrialState.COMPLETE: 1>, value=0.972027972027972, datetime_start=datetime.datetime(2019, 11, 1, 16, 1, 6, 594306), datetime_complete=datetime.datetime(2019, 11, 1, 16, 1, 6, 732933), params={'booster': 'gbtree', 'lambda': 3.413395266900941e-07, 'alpha': 1.295857102451433e-07, 'max_depth': 5, 'eta': 0.8556609659891082, 'gamma': 4.211119213485256e-07, 'grow_policy': 'depthwise'}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(low=1e-08, high=1.0), 'alpha': LogUniformDistribution(low=1e-08, high=1.0), 'max_depth': IntUniformDistribution(low=1, high=9), 'eta': LogUniformDistribution(low=1e-08, high=1.0), 'gamma': LogUniformDistribution(low=1e-08, high=1.0), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide'))}, user_attrs={}, system_attrs={'_number': 8}, intermediate_values={0: 0.98003, 1: 0.991653, 2: 0.994294, 3: 0.995351, 4: 0.995351, 5: 0.995562, 6: 0.995139, 

In [18]:
def objective2(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

In [19]:
study = optuna.create_study()

In [20]:
study.optimize(objective, n_trials=100)

[0]	validation-auc:0.959742
[1]	validation-auc:0.959003
[2]	validation-auc:0.959003
[3]	validation-auc:0.959108
[4]	validation-auc:0.959108
[5]	validation-auc:0.959003
[6]	validation-auc:0.959108
[7]	validation-auc:0.959108
[8]	validation-auc:0.959003
[9]	validation-auc:0.959108


[I 2019-11-01 16:17:35,931] Finished trial#0 resulted in value: 0.9300699300699301. Current best value is 0.9300699300699301 with parameters: {'booster': 'dart', 'lambda': 2.0242553550480727e-06, 'alpha': 0.0006309054926632025, 'max_depth': 7, 'eta': 4.9395214207497665e-06, 'gamma': 3.260977886741957e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 6.964040651093411e-06, 'skip_drop': 1.4762033396620063e-06}.


[0]	validation-auc:0.921247
[1]	validation-auc:0.922441
[2]	validation-auc:0.922441
[3]	validation-auc:0.922441
[4]	validation-auc:0.922441
[5]	validation-auc:0.922441
[6]	validation-auc:0.922441
[7]	validation-auc:0.922441
[8]	validation-auc:0.92284
[9]	validation-auc:0.92284


[I 2019-11-01 16:17:36,062] Finished trial#1 resulted in value: 0.8881118881118881. Current best value is 0.8881118881118881 with parameters: {'booster': 'dart', 'lambda': 0.004171898819371491, 'alpha': 2.3929910118714615e-05, 'max_depth': 9, 'eta': 4.1211137779247256e-05, 'gamma': 3.1004592521167977e-07, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.00028213277888973726, 'skip_drop': 4.440308161332097e-06}.


[0]	validation-auc:0.641772
[1]	validation-auc:0.421624
[2]	validation-auc:0.722319
[3]	validation-auc:0.77703
[4]	validation-auc:0.825445
[5]	validation-auc:0.838689
[6]	validation-auc:0.848459
[7]	validation-auc:0.86561
[8]	validation-auc:0.879288
[9]	validation-auc:0.888841


...

[I 2019-11-01 16:17:50,614] Setting status of trial#97 as TrialState.PRUNED. Trial was pruned at iteration 1.


[0]	validation-auc:0.900726


[I 2019-11-01 16:17:50,770] Setting status of trial#98 as TrialState.PRUNED. Trial was pruned at iteration 1.


[0]	validation-auc:0.932341


[I 2019-11-01 16:17:50,921] Setting status of trial#99 as TrialState.PRUNED. Trial was pruned at iteration 1.


In [21]:
study.best_params

{'booster': 'gblinear',
 'lambda': 1.730669410884869e-07,
 'alpha': 1.1351103845586497e-05}